In [45]:
import tensorflow as tf
import numpy as np

In [46]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [47]:
import deepdish as dd

In [48]:
mydata = dd.io.load("fmnist_simple.h5")
mydata.keys()

dict_keys(['model_weights', 'backend', 'keras_version', 'model_config'])

In [49]:
def serialize_example(example):
    """Serialize an item in a dataset
    Arguments:
      example {[list]} -- list of dictionaries with fields "name" , "_type", and "data"

    Returns:
      [type] -- [description]
    """
    dset_item = {}
    for feature in example.keys():
        dset_item[feature] = example[feature]["_type"](example[feature]["data"])
        example_proto = tf.train.Example(features=tf.train.Features(feature=dset_item))
    return example_proto.SerializeToString()

In [50]:
from tqdm.autonotebook import tqdm # just a progressbar
n_observations = len(mydata["float32_labels"])  # how many items are in your dataset
# loop through hdf5 of examples, save to tfrecord
with tf.io.TFRecordWriter(str('myfile.tfrecord')) as writer:
    # for each example
    for exi in tqdm(range(n_observations)):
        # create an item in the datset converted to the correct formats (float, int, byte)
        example = serialize_example(
            {
                "float32_labels": {
                    "data": mydata["float32_labels"][exi],
                    "_type": _float_feature,
                },
                "int64_labels": {
                    "data": mydata["int64_labels"][exi],
                    "_type": _int64_feature,
                },
                "text_labels": {
                    "data": np.string_(mydata["text_labels"][exi]).astype("|S7"),
                    "_type": _bytes_feature,
                },
                "train_images": {
                    "data": mydata["train_images"][exi].flatten().tobytes(),
                    "_type": _bytes_feature,
                },
            }
        )
        # write the defined example into the dataset
        writer.write(example)

KeyError: 'float32_labels'